<a href="https://colab.research.google.com/github/arunimamuralitharan/aitask/blob/main/querysum_yake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pymupdf yake nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 11.6 MB/s eta 0:00:00


In [14]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import fitz
from google.colab import files
import re
import yake

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [3]:
uploaded=files.upload()

Saving invoice_10248.pdf to invoice_10248.pdf
Saving invoice_10249.pdf to invoice_10249.pdf
Saving invoice_10250.pdf to invoice_10250.pdf
Saving invoice_10251.pdf to invoice_10251.pdf
Saving invoice_10252.pdf to invoice_10252.pdf
Saving invoice_10253.pdf to invoice_10253.pdf
Saving invoice_10254.pdf to invoice_10254.pdf
Saving invoice_10255.pdf to invoice_10255.pdf
Saving invoice_10256.pdf to invoice_10256.pdf
Saving invoice_10257.pdf to invoice_10257.pdf
Saving invoice_10258.pdf to invoice_10258.pdf
Saving invoice_10259.pdf to invoice_10259.pdf
Saving invoice_10260.pdf to invoice_10260.pdf
Saving invoice_10261.pdf to invoice_10261.pdf
Saving invoice_10262.pdf to invoice_10262.pdf
Saving invoice_10263.pdf to invoice_10263.pdf
Saving invoice_10264.pdf to invoice_10264.pdf
Saving invoice_10265.pdf to invoice_10265.pdf
Saving invoice_10266.pdf to invoice_10266.pdf
Saving invoice_10267.pdf to invoice_10267.pdf
Saving invoice_10268.pdf to invoice_10268.pdf
Saving invoice_10269.pdf to invoic

In [4]:
pdf_texts = []
filenames = list(uploaded.keys())

for file in filenames:
    doc = fitz.open(file)
    text = ""
    for page in doc:
        text += page.get_text()
    pdf_texts.append(text)

In [5]:
def extract_keywords_with_yake(pdf_texts, max_keywords=20):
    kw_extractor = yake.KeywordExtractor(lan="en", n=3, top=max_keywords)
    all_keywords = []

    for text in pdf_texts:
        keywords = kw_extractor.extract_keywords(text)
        cleaned = [kw for kw, _ in keywords]
        all_keywords.append(cleaned)

    return all_keywords

yake_keywords = extract_keywords_with_yake(pdf_texts)

In [30]:
def answer_invoice_query(query: str, pdf_texts, filenames, yake_keywords):
    tokens = word_tokenize(query)
    tagged = pos_tag(tokens)

    id_match = re.search(r"\b(\d{4,6})\b", query)
    if not id_match:
        return "Couldn't find Invoice ID match"
    order_id = id_match.group(1)

    lower_query = query.lower()
    intent = None

    if "country" in lower_query:
        intent = "country"
    elif "date" in lower_query:
        intent = "date"
    elif "name" in lower_query or "customer" in lower_query or "contact" in lower_query:
        intent = "customer_name"
    elif "price" in lower_query or "amount" in lower_query or "total" in lower_query:
        intent = "total_price"
    elif "product id" in lower_query or "product number" in lower_query:
        intent = "product_id"
    elif "product" in lower_query or "item" in lower_query:
        intent = "products"
    elif "phone" in lower_query:
        intent = "phone"
    elif "fax" in lower_query:
        intent = "fax"
    elif "summary" in lower_query or "keywords" in lower_query:
        intent = "keywords"


    for i, text in enumerate(pdf_texts):
        if f"Order ID: {order_id}" in text:
            if intent == "customer_name":
                match = re.search(r"Contact Name:\s*(.+)", text)
                return match.group(1).strip() if match else "Customer name not found."

            elif intent == "total_price":
                match = re.search(r"TotalPrice\s*([\d.]+)", text)
                return match.group(1) if match else "Total price not found."

            elif intent == "date":
                match = re.search(r"Order Date:\s*(.+)", text)
                return match.group(1).strip() if match else "Order date not found."

            elif intent == "products":
                product_lines = re.findall(r"(\d+)\s+(.+?)\s+(\d+)\s+([\d.]+)", text)
                return ", ".join([p[1].strip() for p in product_lines]) if product_lines else "No products found."

            elif intent == "product_id":
                product_lines = re.findall(r"(\d+)\s+(.+?)\s+(\d+)\s+([\d.]+)", text)
                for p in product_lines:
                    product_name = p[1].strip().lower()
                    for kw in yake_keywords[i]:
                        if product_name in lower_query and product_name in kw.lower():
                            return p[0]
                return "Couldn't find matching product for Product ID."

            elif intent == "country":
                match = re.search(r"Country:\s*(.+)", text)
                return match.group(1).strip() if match else "Country not found."

            elif intent == "phone":
                match = re.search(r"Phone:\s*(.+)", text)
                return match.group(1).strip() if match else "Phone number not found."

            elif intent == "fax":
                match = re.search(r"Fax:\s*(.+)", text)
                return match.group(1).strip() if match else "Fax number not found."

            elif intent == "keywords":
                return ", ".join(yake_keywords[i]) if yake_keywords[i] else "No keywords extracted."

            else:
                return "Found order, but couldn't understand the request."

    return f"Order ID {order_id} not found."

In [31]:
while True:
    query = input("Ask your invoice question (or type 'exit'): ")
    if query.lower() == "exit":
        break
    result = answer_invoice_query(query, pdf_texts, filenames,yake_keywords)
    print("→", result)

Ask your invoice question (or type 'exit'): What country was the order for invoice 10248 sent to?
→ France
Ask your invoice question (or type 'exit'): what products are there for 10248
→ Queso Cabrales, Singaporean Hokkien Fried Mee, Mozzarella di Giovanni
Ask your invoice question (or type 'exit'): product id of queso cabrales in 10248
→ 11
Ask your invoice question (or type 'exit'): exit


In [32]:
test_queries = [
    "What is the total price in invoice 10248?",
    "Who is the customer for invoice 10248?",
    "What is the product ID of Mozzarella di Giovanni in invoice 10248?",
    "What country was the order for invoice 10248 sent to?",
    "Give me the keywords for invoice 10248"
]

ground_truth = [
    "440.0",
    "Paul Henriot",
    "72",
    "France",
    "mozarella di giovanni"
]

In [33]:
from difflib import SequenceMatcher

def is_close_match(a, b, threshold=0.9):
    return SequenceMatcher(None, a.lower().strip(), b.lower().strip()).ratio() >= threshold

def evaluate_accuracy_yake(test_queries, ground_truth, pdf_texts, filenames, yake_keywords):
    correct = 0
    total = len(test_queries)

    print("\n Evaluating...\n")
    for i, (query, expected) in enumerate(zip(test_queries, ground_truth)):
        predicted = answer_invoice_query(query, pdf_texts, filenames, yake_keywords)
        if is_close_match(predicted, expected):
            correct += 1
        else:
            print(f" {i+1}. Query: {query}")
            print(f"   Expected: {expected}")
            print(f"   Got     : {predicted}\n")

    accuracy = correct / total
    print(f"\n Accuracy: {accuracy*100:.2f}% ({correct}/{total} correct)")
    return accuracy

In [34]:
evaluate_accuracy_yake(test_queries, ground_truth, pdf_texts, filenames, yake_keywords)


 Evaluating...

 5. Query: Give me the keywords for invoice 10248
   Expected: mozarella di giovanni
   Got     : Paul Henriot Address, Reims Postal Code, VINET Order Date, Quantity Unit Price, Singaporean Hokkien Fried, Hokkien Fried Mee, France Phone, Queso Cabrales, Mozzarella di Giovanni, Customer Details, Paul Henriot, Henriot Address, Reims Postal, Postal Code, Unit Price, Singaporean Hokkien, Fried Mee, VINET Order, Order Date, Quantity Unit


 Accuracy: 80.00% (4/5 correct)


0.8